<a href="https://colab.research.google.com/github/DuplamenteH/Ds/blob/master/Series_Temporais_com_Prophet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Previsão de Séries Temporais com o Prophet

O projeto trata-se da previsão de outcomes de animais de um abrigo.

Como ferramente vamos utilizar o Prophet lib muito famosa e poderosa para lidar com TS(Series Temporais), o framework Prophet é disponibilizado como projeto open source.

Aqui esta o [artigo](https://peerj.com/preprints/3190v2/).

Resumidamente o Prophet é um framework de forecasting para series temporais, implementado para as linguagens Python e R é capaz de fazer previsões automatizadas e muito mais rápidas.

[**Documentação Oficial.**](https://facebook.github.io/prophet/).

Por ser um Framework open-source é possivel que Cientistas de dados otimizem parâmetros do framework.

## Modelo Preditivo do Prophet:

O Prophet foi feito para solucionar problemas do Facebook. Ou seja, é muito forte para lidar com problemas que tenham características como:

* Dados coletados por um longo período (meses ou anos), e que possuam o maior detalhamento histórico possível.
* Sazonalidades fortes e destacadas.
* Feriados ou datas especiais conhecidas *a priori*.
* Tendência de crescimento não linear, que se aproxima de um limite.

Propeht usa um modelo de séries temporais decomposto, com três componentes principais: tendência, sazonalidade e feriados:

$$
y(t) = g(t) + s(t) + h(t) + \varepsilon_t
$$

onde:

* $g(t)$ é a tendência, que modela as mudanças não-periódicas no valor da série temporal;
* $s(t)$ representa as mudanças periódicas (como sazonalidade semanal, anual, etc);
* $h(t)$ representa os efeitos dos feriados; e
* $\varepsilon_t$ é o erro, as mudanças que não são capturadas pelo modelo (assume-se esse como tendo uma distribuição normal).




# O Problema:

## Abrigo de animais

O centro de animais de Austin, no Texas, provê  abrigo para mais de 16 mill animais por ano, além de outros serviços relacionados a saúde dos animais, independente da raça, idade ou estado de saúde dos mesmos.

O objetivo principal do abriga é fazer com que os animais recebam novos lares, seja por adoção, resgate  ou ação social.

Porém por maior que seja os esforços do abrigo ainda assim não é possível dar lar a todos os animais, muitos desses animais acabam sendo sacrificados.O abrigo Austin animal Center tem como meta conseguir que 90% dos animais que vivem no abrigo tenham um final feliz.

Nossa função aqui é anilizar as saídas dessas animais, com essa análise , espera-se que sejamos capazes de indenitifcar um padrão de adoção.Também esperamos conseguir fazer previsões futuras em relação ao número de saída do abrigo, tendo como objetivo:


1. Planejar compras de insumos(remédios, comida, etc);
2. Contratações de funcionários e colaboradores;
3. Fazer campanhas visando aumentar a adoção de animais.







## Os dados.
Os dados foram disponibilizados pelo [Austin Animal Center](http://www.austintexas.gov/content/austin-animal-center) e podem ser baixados diretamente no [City of Austin Open Data Portal](https://data.austintexas.gov/Health-and-Community-Services/Austin-Animal-Center-Outcomes/9t4d-g238).

In [ ]:
#instalações
!pip install plotly==4.14.3


In [ ]:
!pip install biokit


In [ ]:
#importações  de bibliotecas necessárias
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from fbprophet import Prophet
from fbprophet.plot import add_changepoints_to_plot
from fbprophet.diagnostics import cross_validation
from fbprophet.diagnostics import performance_metrics
from fbprophet.plot import plot_cross_validation_metric
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

import plotly.graph_objects as go
from biokit.viz import corrplot
import plotly.express as px


# configurar a visualização
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [ ]:
# importar os dados para um DataFrame
df = pd.read_csv("https://github.com/carlosfab/escola-data-science/blob/master/datasets/shelter_animal_outcomes.csv?raw=true")

# converter a coluna DateTime para o formato conveniente
df.DateTime = pd.to_datetime(df.DateTime)

# ver as primeiras 5 entradas
df.head()

## Análise dos dados

Resumo das estatísticas

In [ ]:
df.describe()

vemos que existem muitos dados NaN

In [ ]:
df.info()

In [ ]:
dfcopia = df.copy()
dfcopia.head()

In [ ]:
colunas=['Outcome Subtype', 'Name']

df.drop(columns=colunas, inplace=True)
df.head()

In [ ]:
(df['Outcome Type'].value_counts()/df.shape[0])*100

In [ ]:
adotados = df[df['Outcome Type']=='Adoption']
transferidos = df[df['Outcome Type']=='Transfer']
mortos_ou_eutanasia =  df.loc[(df['Outcome Type']=='Died') | (df['Outcome Type']=='Euthanasia')]
disponiveis = df[df['Outcome Type']=='Disposal']
perdidos = df[df['Outcome Type']=='Missing']
realocados = df[df['Outcome Type']=='Relocate']
Rtn_dono =df[df['Outcome Type']=='Return to Owner']
rtn_adocao=df[df['Outcome Type']=='Rto-Adopt']



Criando um dict com os dados acima, para fazer a plotagem desses dados.

In [ ]:
dados = {
    'adotados':[adotados.shape[0]],
    'transferidos':[transferidos.shape[0]],
    'mortos_ou_eutanasia':[mortos_ou_eutanasia.shape[0]],
    'disponiveis':[disponiveis.shape[0]],
    'perdidos':[perdidos.shape[0]],
    'realocados':[realocados.shape[0]],
    'Rtn_dono':[Rtn_dono.shape[0]],
    'rtn_adocao':[rtn_adocao.shape[0]]
}
dados

Criando um df com o dict de cima

In [ ]:
outcomesDF=pd.DataFrame(data=dados,index=['valor'])

In [ ]:
outcomesDF

In [ ]:
outcomesDF.columns

Plotagem dos dados.

In [ ]:
fig = go.Figure(data=[
    go.Bar(name='adotados', x=['adotados'], y=outcomesDF.adotados),
    go.Bar(name='transferido', x=['transferidos'], y=outcomesDF.transferidos),
    go.Bar(name='Rtn_dono', x=['Rtn_dono'], y=outcomesDF.Rtn_dono), 
    go.Bar(name='mortos_ou_eutanasia', x=['mortos_ou_eutanasia'], y=outcomesDF.mortos_ou_eutanasia),
    go.Bar(name='rtn_adocao', x=['rtn_adocao'], y=outcomesDF.rtn_adocao),
    go.Bar(name='Adotadisponiveisdos', x=['disponiveis'], y=outcomesDF.disponiveis),
    go.Bar(name='realocados', x=['realocados'], y=outcomesDF.realocados),
    
])
# Change the bar mode
fig.update_layout(barmode='group')
fig.show()

In [ ]:
percentis = (df['Outcome Type'].value_counts()/df.shape[0])*100
percentis


In [ ]:
labels = ['Adotados', 'Transferidos', 'Rtn_dono',' Eutanasia','Mortos','Rtn Adocao','Disponiveis para adoção','Perdidos','Realocados']
values = df['Outcome Type'].value_counts()

fig = go.Figure(data=[go.Pie(labels=labels, values=values, textinfo='percent', insidetextorientation='radial')])
fig.show()

### Observando os gráficos , vemos que o abrigo está fazendo com que a maioria dos animais saia do abrigo com vida e com um dono.

## Qual raça mais adotada de gatos e cachorros?

In [ ]:
dogs = adotados[adotados['Animal Type']=='Dog']
(dogs['Breed'].value_counts()/dogs.shape[0])*100

In [ ]:
cats = adotados[adotados['Animal Type']=='Cat']
(cats['Breed'].value_counts()/cats.shape[0])*100

### Entre os cachorros a disputa é bem acirrada entre o labrador com **10,85%** de adotados contra o o Pit bull Mix com **10,31%**. Já nos gatos temos uma vasta diferença entre o primeiro e o segundo colocado com **68,72%** o primeiro colocado é Domestic Shorthair Mix é a raça de gato mais adotada e  em segundo lugar com **11,23%** é o Domestic Shorthair.

## Feature Eng.

### Nesta parte vamos criar uma nova feature para categoria de adotados, para possamos fazer uma contagem de adotados.

In [ ]:
adotados['outcomeCount']=1


In [ ]:
adotados.head()

### Com isso agora conseguimos fazer nossa contagem, agora separar apenas as partes que nos interessa por enquanto. Vamos criar um novo df com nome  *adotTS* onde irá conter apenas a data e nossa nova feature outcomeCount.Nesse df se tornará uma serie temporal, onde o DataTime será nosso index

In [ ]:
adotTS = adotados[['DateTime','outcomeCount']]
adotTS = adotTS.set_index('DateTime').sort_index()
adotTS.head()

In [ ]:
adotados.index

In [ ]:
#grafico Diario
fig, ax = plt.subplots(figsize=(15,7.5))
adotTS.resample('D').sum().plot(ax=ax)
plt.show()

Com o grafico diario fica muito conturbado tirar alguma informação.

In [ ]:
#Grafico mensal
fig, ax = plt.subplots(figsize=(15,7.5))
adotTS.resample('M').sum().plot(ax=ax)
plt.show()

In [ ]:
#Grafico anual
fig, ax = plt.subplots(figsize=(15,7.5))
adotTS.resample('Y').sum().plot(ax=ax)
plt.show()

## Modelos com o Prophet

Primeiro definimos um objeto *Prophet()* e depois para treinar usamos
o metodo *fit()* no metodo passaremos o nosso DF que ter configurado a primeira coluna deve ter o nome **ds** e ira conter os horários de data. A segunda coluna deve ter o nome  **y** e deve ter os valores..


~~~Python
modelo = Prophet()
modelo.fit(df)
~~~

### Modelo Diario.

In [ ]:
"""

        Modificação do Dataset.


"""

prophet_df = adotTS.resample('D').sum().reset_index().rename(columns = {'DateTime':'ds', 'outcomeCount':'y'})

In [ ]:
prophet_df

In [ ]:
"""

    Criação, treino e plotagem de componentes nas próximas 3 células.

"""
model = Prophet()
model.add_country_holidays(country_name='US')
model.fit(prophet_df)

#periodo da previsão
future = model.make_future_dataframe(periods = 365)

#realizar a previsão para o período estabelecido
forecast = model.predict(future)

In [ ]:
model.plot(forecast, xlabel='Data', ylabel='Adoções');

In [ ]:
# separar os componentes do modelo
model.plot_components(forecast);

Vemos que nos sábados tem uma taxa mais alta de adoções, e as taxas mais baixas é na , e o mês que mais tem adoção é um julho.

### Modelo semanal

In [ ]:
prophet_df2 = adotTS.resample('W').sum().reset_index().rename(columns = {'DateTime':'ds', 'outcomeCount':'y'})
model2 = Prophet()
model2.add_country_holidays(country_name='US')
model2.fit(prophet_df2)

#periodo da previsão
future2 = model2.make_future_dataframe(periods = 52)

#realizar a previsão para o período estabelecido
forecast2 = model2.predict(future2)

In [ ]:
# visualizar o gráfico com as previsões
model2.plot(forecast2, xlabel='Data', ylabel='Adoções');

In [ ]:
# separar os componentes do modelo
model2.plot_components(forecast2);

# Avaliação dos modelos

## Modelo diario

In [ ]:
from fbprophet.diagnostics import cross_validation
df_cv = cross_validation(model, horizon = '365 days')
df_cv.head()

In [ ]:
from fbprophet.diagnostics import performance_metrics
df_p = performance_metrics(df_cv)
df_p.head()

In [ ]:
from fbprophet.plot import plot_cross_validation_metric
fig = plot_cross_validation_metric(df_cv, metric='mdape')

In [ ]:
fig = plot_cross_validation_metric(df_cv, metric='mae')

In [ ]:
fig = plot_cross_validation_metric(df_cv, metric='rmse')

In [ ]:
fig = plot_cross_validation_metric(df_cv, metric='mse')

In [ ]:
fig = plot_cross_validation_metric(df_cv, metric='coverage')

## Modelo semanal

In [ ]:
from fbprophet.diagnostics import cross_validation
df_cv2 = cross_validation(model2, horizon = '365 days')

from fbprophet.diagnostics import performance_metrics
df_p2 = performance_metrics(df_cv2)

;

In [ ]:
df_p2.head()

In [ ]:
df_p2.tail()

In [ ]:
from fbprophet.plot import plot_cross_validation_metric
plot_cross_validation_metric(df_cv2, metric='mape');

In [ ]:
plot_cross_validation_metric(df_cv2, metric='mse');

In [ ]:
plot_cross_validation_metric(df_cv2, metric='mae');

In [ ]:
plot_cross_validation_metric(df_cv2, metric='mdape');

In [ ]:
plot_cross_validation_metric(df_cv2, metric='rmse');